In [ ]:
pip install cryptography

Необхідно:

1. Згенерувати довгострокову ключову пару для підпису алгоритмом ECDSA на кривій SECP256K1.
2. Згенерувати приватний ключ боба для узгодження ключа алгоритмом X25519.
3. Перевірити підпис відкритого ключа Alice для узгодження ключа (використовуючи відкритий ключ Alice для підпису).
4. Створити відкритий ключ ECDH (значення Y = yP) для надсилання Alice
5. Cтворити підпис значення Y використовуючи приватний довгостроковий ключ Боба для підпису.

In [3]:
path = "/content/drive/MyDrive/robot_dreams_cryptography/HW8/"


from binascii import hexlify, unhexlify
from cryptography.hazmat.primitives.asymmetric import ec
from cryptography.hazmat.primitives.asymmetric.x25519 import (
    X25519PrivateKey, X25519PublicKey
)
from cryptography.hazmat.primitives import hashes, serialization
from cryptography.hazmat.primitives.kdf.hkdf import HKDF
from cryptography.hazmat.primitives.serialization import Encoding, PublicFormat
from cryptography.exceptions import InvalidSignature


# 1. Генерація довгострокової ключової пари Боба алгоритмом ECDSA (SECP256K1) для підпису
bob_sign_private_key = ec.generate_private_key(ec.SECP256K1())
bob_sign_public_key = bob_sign_private_key.public_key()

#Збереження відкритого ключа ECDSA Боба у форматі PEM
bob_sign_pub_key_pem = bob_sign_public_key.public_bytes(
    Encoding.PEM,
    PublicFormat.SubjectPublicKeyInfo,
)

with open(path + "bob_sign_pub_key.pem", "wb") as f:
    f.write(bob_sign_pub_key_pem)

print("Bob signing public key (PEM):\n", bob_sign_pub_key_pem.decode())


# 2. + 4. Генерація ключової пари алгоритмом ECDH (X25519) Боба та створення відкритого ключа ECDH Боба
bob_ecdh_private_key = X25519PrivateKey.generate()
bob_ecdh_public_key = bob_ecdh_private_key.public_key()

#Збереження відкритого ключа ECDH Боба у форматі Raw розміром 32 байти (по аналогії з ключем Аліси)
bob_ecdh_public_bytes = bob_ecdh_public_key.public_bytes(
    Encoding.Raw,
    PublicFormat.Raw,
)
print("\nBob's ECDH public key (hex):", hexlify(bob_ecdh_public_bytes).decode())


#Завантаження вхідних даних:

#Завантаження відомого публічного ключа Аліси та її підпису
alice_pub_sign_key_raw = b"""
-----BEGIN PUBLIC KEY-----
MFYwEAYHKoZIzj0CAQYFK4EEAAoDQgAES/35y89DRx2XEh6pJvCckadQ9Awuys84
HORPVVaDksVxWfSkngYrz/c+HwVS9tV5ivnVwCHxyJ8gTQob/0LDDg==
-----END PUBLIC KEY-----
"""

#Завантаження відкритого ключа Аліси з формату PEM
alice_pub_sign_key = serialization.load_pem_public_key(alice_pub_sign_key_raw)

#Перетворення у байти для можливості перевірки ЕП
alice_x_pub_key_bytes = unhexlify(b'92ce3bc6d941238da92639c72a7d3bb483d3c18fdca9f42164459a3751638433')
alice_signature = unhexlify(b'3045022034b7944bf92bfaa2791b5fe929d915add4ee59dbd9e776c1520568fbf2503048022100f09c9113f38fadb33b05332eab9a4982f7dda35fb1f503bb46da806c8e8dbaa2')


# 3. Перевірка підпису Аліси (ECDSA)
try:
    alice_pub_sign_key.verify(
        alice_signature,
        alice_x_pub_key_bytes,
        ec.ECDSA(hashes.SHA256())
    )
    print("\nAlice's signature verified successfully.")
except InvalidSignature:
    print("\nAlice's signature verification failed!")
    exit(1)  #У випадку помилки при перевірці ЕП весь процес зупиняємо


# 5. Підпис відкритого ключа ECDH Боба (ECDSA)
bob_signature = bob_sign_private_key.sign(
    bob_ecdh_public_bytes,
    ec.ECDSA(hashes.SHA256())
)

print("\nBob's ECDH public key signature (hex):", hexlify(bob_signature).decode())


# Створення спільного секрету і ключа (ECDH) /сторона Боба/
alice_ecdh_public_key = X25519PublicKey.from_public_bytes(alice_x_pub_key_bytes)
shared_secret = bob_ecdh_private_key.exchange(alice_ecdh_public_key)

derived_key = HKDF(
    algorithm=hashes.SHA256(),
    length=32,
    salt=None, # Важливо не додавати рандомізацію для отримання однакового ключа з обох сторін
    info=b'handshake data'
).derive(shared_secret)

print("\nBob's derived secret key (hex):", hexlify(derived_key).decode())

Bob signing public key (PEM):
 -----BEGIN PUBLIC KEY-----
MFYwEAYHKoZIzj0CAQYFK4EEAAoDQgAEedNHl5hOF4Wi7iV77BYPMDSiUmh43DV6
tRGDCDbAFbtYjgvFMtS/KZFneFQybQxlCaD0V0UNKpCfwoRmvVik8g==
-----END PUBLIC KEY-----


Bob's ECDH public key (hex): 9b8e148f93756f6685d55f493594d6aa8b9d278872e8fef8f4fa42b56a136136

Alice's signature verified successfully.

Bob's ECDH public key signature (hex): 304402204664e3d80365c03b5551832db13d9469a8c165164d6b912026600e6156b7ef6902200e5911edcadb6eacac366a694509c666cd6b4e3fa258948cf342673e46510d92

Bob's derived secret key (hex): 1fa2d9e7572b17e8cd54de03f8c255818973945b080449c075a85412936247df
